## T2310: TCS3351 Cryptography and Data Security - Assignment (40%)


##### Step 1 - Person B - Generate RSA Key Pair.txt

In [ ]:
from Crypto.PublicKey import RSA

# Generate RSA key pair
key_pair = RSA.generate(2048)

# Export private key
private_key = key_pair.export_key()
with open('private_key.pem', 'wb') as private_file:
    private_file.write(private_key)

# Export public key
public_key = key_pair.publickey().export_key()
with open('public_key.pem', 'wb') as public_file:
    public_file.write(public_key)

    The code imports the necessary module RSA from the Crypto.PublicKey package.

    The RSA key pair is generated using the RSA.generate() method, and it is stored in the key_pair variable.

    The private key is exported from the key_pair using the export_key() method, and the result is stored in the private_key variable.

    The private key is then written to a file named 'private_key.pem' in binary mode using a with statement and the open() function. The file object is assigned to the variable private_file.

    The write() method is used to write the private_key content to the private_file.

    Similarly, the public key is obtained from the key_pair using the publickey() method, and it is exported using the export_key() method. The result is stored in the public_key variable.

    The public key is written to a file named 'public_key.pem' in binary mode using a with statement and the open() function. The file object is assigned to the variable public_file.

    The write() method is used to write the public_key content to the public_file.

##### Step 2 - Person A - Encrypting Key and Plaintext

In [ ]:
import os
import base64
from Crypto.Cipher import AES, PKCS1_OAEP
from Crypto.Random import get_random_bytes
from Crypto.PublicKey import RSA

def encrypt_message(message, key):
    # Create an AES cipher object with the key and CTR mode
    aes_cipher = AES.new(key, AES.MODE_CTR)

    # Encrypt the message
    ciphertext = aes_cipher.encrypt(message)
    return ciphertext, aes_cipher.nonce

def export_rsa_encrypted_key(aes_key, rsa_public_key, output_file):
    # Create a PKCS1_OAEP cipher object with the RSA public key
    rsa_cipher = PKCS1_OAEP.new(rsa_public_key)

    # Encrypt the AES key using RSA encryption
    encrypted_key = rsa_cipher.encrypt(aes_key)

    # Write the encrypted key to the output file
    with open(output_file, 'wb') as file:
        file.write(encrypted_key)

# Person A inputs the message
message = input("Enter the message: ").encode()

# Generate AES key
aes_key = get_random_bytes(32)

# Encrypt the message using AES in CTR mode
ciphertext, nonce = encrypt_message(message, aes_key)

# Export the encrypted AES key using RSA
rsa_public_key = RSA.import_key(open('public_key.pem').read())
export_rsa_encrypted_key(aes_key, rsa_public_key, 'encrypted_key.bin')

# Export the ciphertext and nonce to files
with open('ciphertext.bin', 'wb') as file:
    file.write(ciphertext)

with open('nonce.bin', 'wb') as file:
    file.write(nonce)

# Print the encrypted ciphertext and encoded AES key
print("Encrypted Ciphertext:", base64.b64encode(ciphertext).decode())
print("Encoded AES Key:", base64.b64encode(aes_key).decode())


    The code imports necessary modules from the Crypto library and other standard libraries.

    The encrypt_message() function takes a message and an AES key as input. It creates an AES cipher object using the key and CTR mode. The message is then encrypted using the cipher, and the ciphertext and nonce (generated by the AES cipher) are returned.

    The export_rsa_encrypted_key() function takes an AES key, an RSA public key, and an output file path as input. It creates a PKCS1_OAEP cipher object using the RSA public key. The AES key is encrypted using RSA encryption, and the encrypted key is written to the output file.

    The user is prompted to input a message.

    An AES key is generated using get_random_bytes() function, and it is stored in the aes_key variable.

    The message is encrypted using the encrypt_message() function, passing the message and AES key. The resulting ciphertext and nonce are assigned to the variables ciphertext and nonce, respectively.

    The RSA public key is imported from the 'public_key.pem' file.

    The export_rsa_encrypted_key() function is called to export the encrypted AES key, passing the AES key, RSA public key, and the output file path.

    The ciphertext and nonce are written to separate files.

    The base64-encoded versions of the ciphertext and AES key are printed.

##### Step 3 - Person B - Decrypting Key and Plaintext

In [ ]:
import base64
from Crypto.Cipher import AES, PKCS1_OAEP
from Crypto.PublicKey import RSA

def import_rsa_private_key(private_key_file):
    # Import the RSA private key from a file
    private_key = RSA.import_key(open(private_key_file).read())
    return private_key

def decrypt_aes_key(encrypted_key, rsa_private_key):
    # Create a PKCS1_OAEP cipher object with the RSA private key
    rsa_cipher = PKCS1_OAEP.new(rsa_private_key)

    # Decrypt the AES key using RSA decryption
    aes_key = rsa_cipher.decrypt(encrypted_key)

    return aes_key

def decrypt_message(ciphertext, key, nonce):
    # Create an AES cipher object with the key, CTR mode, and nonce
    aes_cipher = AES.new(key, AES.MODE_CTR, nonce=nonce)

    # Decrypt the ciphertext
    decrypted_message = aes_cipher.decrypt(ciphertext)

    return decrypted_message

# Person B imports the encrypted AES key
encrypted_key = open('encrypted_key.bin', 'rb').read()

# Person B imports the private key
private_key = import_rsa_private_key('private_key.pem')

# Decrypt the AES key using RSA private key
aes_key = decrypt_aes_key(encrypted_key, private_key)

# Person B decrypts the ciphertext using the AES key
ciphertext = open('ciphertext.bin', 'rb').read()
nonce = open('nonce.bin', 'rb').read()

decrypted_message = decrypt_message(ciphertext, aes_key, nonce)

print("Decrypted Message:", decrypted_message.decode())


    The code imports necessary modules from the Crypto library and the base64 module.

    The import_rsa_private_key() function takes a private key file path as input. It reads the private key from the file and imports it using RSA.import_key(). The imported private key is then returned.

    The decrypt_aes_key() function takes an encrypted AES key and an RSA private key as input. It creates a PKCS1_OAEP cipher object using the RSA private key. The encrypted AES key is decrypted using RSA decryption, and the decrypted AES key is returned.

    The decrypt_message() function takes a ciphertext, an AES key, and a nonce as input. It creates an AES cipher object using the key, CTR mode, and the nonce. The ciphertext is decrypted using the cipher, and the decrypted message is returned.

    Person B imports the encrypted AES key from the 'encrypted_key.bin' file.

    Person B imports the private key from the 'private_key.pem' file using the import_rsa_private_key() function.

    The decrypt_aes_key() function is called to decrypt the AES key, passing the encrypted key and the private key. The decrypted AES key is assigned to the variable aes_key.

    Person B reads the ciphertext and the nonce from their respective files.

    The decrypt_message() function is called to decrypt the ciphertext, passing the ciphertext, AES key (aes_key), and nonce. The decrypted message is assigned to the variable decrypted_message.

    The decrypted message is printed.